In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your c_syntax_dataset.json


Saving c_syntax_dataset.json to c_syntax_dataset.json


In [ ]:
import json
from datasets import Dataset

# Load the JSON file
with open("c_syntax_dataset.json") as f:
    data = json.load(f)

# Convert to HuggingFace dataset
dataset = Dataset.from_list(data)

# Train/Validation split
dataset = dataset.train_test_split(test_size=0.1)
dataset


DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 909
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 101
    })
})

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_length = 512

def tokenize_function(example):
    input_enc = tokenizer(example["input"], padding="max_length", truncation=True, max_length=max_length)
    output_enc = tokenizer(example["output"], padding="max_length", truncation=True, max_length=max_length)

    input_enc["labels"] = output_enc["input_ids"]
    return input_enc

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["input", "output"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Map:   0%|          | 0/909 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./codet5-finetuned-cfix",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False
)
print("✅ TrainingArguments created successfully!")


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)


✅ TrainingArguments created successfully!


<ipython-input-5-46a4132754c9>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pandeynavendu15 (pandeynavendu15-graphic-era-hill-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.034200,0.007599
2,0.010700,0.002846
3,0.006400,0.001869
4,0.004200,0.001501
5,0.003600,0.001389


TrainOutput(global_step=1140, training_loss=0.04325280524136727, metrics={'train_runtime': 269.6492, 'train_samples_per_second': 16.855, 'train_steps_per_second': 4.228, 'total_flos': 615128487690240.0, 'train_loss': 0.04325280524136727, 'epoch': 5.0})